<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update Referentials

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook allows to send data to Google Sheets to a Google Sheets spreadsheet.

## Input

### Import libraries

In [1]:
from naas_drivers import gsheet, linkedin
import pandas as pd
import os
from datetime import date
import naas_data_product
import openai
import time
from googlesearch import search
import re
from difflib import SequenceMatcher

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/openai.ipynb' successfully loaded.


### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_reactions`: Name of the file with reactions to be retrieved.
- `file_comments`: Name of the file with comments to be retrieved.

**Outputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `append`: If False, data will be canceled and replaced.

In [2]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
file_name = "linkedin_interactions"
openai_api_key = naas.secret.get("OPENAI_API_KEY")
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585

# Outputs
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
ref_contacts_name = "CONTACTS"
ref_companies_name = "COMPANIES"
leads_profiles_name = "LEADS_PROFILES"
leads_companies_name = "LEADS_COMPANIES"
append = False

## Model

### Get BDD interactions

In [3]:
df_interactions = pload(input_dir, file_name)    
print('🗂️ Interactions:', len(df_interactions))
df_interactions.head(3)

🗂️ Interactions: 1568


,ENTITY,SCENARIO,PLATFORM,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,INTERACTION,INTERACTION_CONTENT,INTERACTION_SCORE,PROFILE_URL,PUBLIC_ID,CONTENT_TITLE,CONTENT_URL,PUBLISHED_DATE,DATE_EXTRACT
0,Jérémy Ravenel,W49-2023,LinkedIn,Abhisek,Nanda,Abhisek Nanda,Product Owner || Business Analyst || Digital S...,POST_REACTION,LIKE,1,https://www.linkedin.com/in/ACoAAAPlOvYBoVWbeK...,abhisek-nanda-a72a7819,Almost every CEOs want to build an internal ch...,https://www.linkedin.com/feed/update/urn:li:ac...,2023-12-04 22:12:53+0100,2023-12-05 11:57:04
1,Jérémy Ravenel,W49-2023,LinkedIn,Abraham,Cherian,Abraham Cherian,Group Solutions Leader @ Brane | TEDx speaker ...,POST_REACTION,LIKE,1,https://www.linkedin.com/in/ACoAAAG1Qd0B4tWJNT...,abrahamcherian,Almost every CEOs want to build an internal ch...,https://www.linkedin.com/feed/update/urn:li:ac...,2023-12-04 22:12:53+0100,2023-12-05 11:57:04
2,Jérémy Ravenel,W49-2023,LinkedIn,Abraham,Cherian,Abraham Cherian,Group Solutions Leader @ Brane | TEDx speaker ...,POST_COMMENT,Great ideas. Some really exciting work already...,3,https://www.linkedin.com/in/ACoAAAG1Qd0B4tWJNT...,abrahamcherian,Almost every CEOs want to build an internal ch...,https://www.linkedin.com/feed/update/urn:li:ac...,2023-12-04 22:12:53+0100,2023-12-05 11:57:07


### Get data "CONTACTS" from Google Sheets spreadsheet

In [4]:
df_contacts = gsheet.connect(spreadsheet_url).get(sheet_name=ref_contacts_name)
print("CRM Contacts:", len(df_contacts))
df_contacts.head(3)

CRM Contacts: 23839


,FIRSTNAME,LASTNAME,FULLNAME,EMAIL,LINKEDIN_URL,COMPANY,CREATEDATE,CONTACT_ID,OWNER_ID,OWNER_EMAIL
0,Kudos,ANASS,Kudos ANASS,kudos95anass@gmail.com,NA,NA,2023-12-05T11:11:08.968Z,776451,NA,NA
1,NA,NA,NA NA,aurelien@lesbellesboites.com,NA,NA,2023-12-05T10:07:57.366Z,776401,158373005,jeremy@naas.ai
2,NA,NA,NA NA,jujudu33700@live.fr,NA,NA,2023-12-05T10:03:28.723Z,776351,NA,NA


### Get data "COMPANIES" from Google Sheets spreadsheet

In [5]:
df_companies = gsheet.connect(spreadsheet_url).get(sheet_name=ref_companies_name)
print("CRM Companies:", len(df_companies))
df_companies.head(3)

CRM Companies: 4199


,NAME,INDUSTRY,COUNTRY,LINKEDINBIO,LINKEDIN_URL,CREATEDATE,CONTACT_ID,OWNER_ID,OWNER_EMAIL
0,Lesbellesboites,NA,France,Les Belles Boîtes is a venture capital and pri...,https://www.linkedin.com/company/les-belles-bo...,2023-12-05T10:07:58.180Z,18320098607,NA,NA
1,Appleid,NA,United States,Apple is a multinational corporation that desi...,https://www.linkedin.com/company/apple,2023-12-04T21:51:42.705Z,18315850819,NA,NA
2,Prompt,NA,NA,NA,https://www.linkedin.com/company/promptb2b,2023-12-04T21:51:41.529Z,18315831358,158373005,jeremy@naas.ai


### Get data "LEADS" from Google Sheets spreadsheet

In [6]:
df_leads = gsheet.connect(spreadsheet_url).get(sheet_name=leads_profiles_name)
print("Leads:", len(df_leads))
df_leads.head(1)

Leads: 848


,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_URL,PUBLIC_ID,INTERACTION_SCORE,ICP,COMPANY,NOTES,CRM_CONTACT_ID,LAST_INTERACTION_DATE,LAST_CONTENT_URL_INTERACTION,LAST_CONTENT_TITLE_INTERACTION
0,Jonathan,La Cruz,Jonathan La Cruz,Information Technology & Business intelligence...,https://www.linkedin.com/in/ACoAAAlYSwQBS9GBiC...,jonathanlacruz,80,DataProducer,Finerio Connect,Like 'Happy Thanksgiving everyone. 🍁🍂🥧' (https...,663201,Sun. 03 Dec.,https://www.linkedin.com/feed/update/urn:li:ac...,"I would even say “Good AI requires clean data,..."


### Get data "LEADS_COMPANIES" from Google Sheets spreadsheet

In [7]:
df_leads_companies = gsheet.connect(spreadsheet_url).get(sheet_name=leads_companies_name)
print("Leads Companies:", len(df_leads_companies))
df_leads_companies.head(1)

Leads Companies: 321


,COMPANY_ID,COMPANY,COMPANY_NAME,LINKEDIN_URL,DIRECT_INTERACTIONS,INDIRECT_INTERACTIONS,INTERACTION_SCORE,INDUSTRY,STAFF_COUNT,STAFF_RANGE,FOLLOWER_COUNT,COUNTRY,CITY,WEBSITE,TAGLINE,DESCRIPTION
0,70506391,naas.ai,naas.ai,https://www.linkedin.com/company/naas-ai,0,61,61,Software Development,22,2-10,2986,FR,Paris,https://www.naas.ai/,⚡️ All-in-one open source data platform\n#lowc...,Naas is the first Jupyter based data-science p...


### Get interactions unique profiles

In [8]:
def get_unique_profile(
    df_init
):
    # Init
    df_profiles = df_init.copy()
    df_last_interaction = df_init.copy()

    # Groupby profile
    to_group = [
        "FIRSTNAME",
        "LASTNAME",
        "FULLNAME",
        "OCCUPATION",
        "PROFILE_URL",
        "PUBLIC_ID"
    ]
    to_agg = {
        "INTERACTION_SCORE": "sum"
    }
    df_profiles = df_profiles.groupby(to_group, as_index=False).agg(to_agg).drop_duplicates("PROFILE_URL")
    df_profiles = df_profiles.sort_values(by="INTERACTION_SCORE", ascending=False).reset_index(drop=True)
    
    # Add last interactions data
    to_keep = [
        "PROFILE_URL",
        "PUBLISHED_DATE",
        "CONTENT_URL",
        "CONTENT_TITLE"
    ]
    df_last_interaction = df_last_interaction[to_keep].drop_duplicates().drop_duplicates(["PROFILE_URL"])
    
    # Merge dfs
    df = pd.merge(df_profiles, df_last_interaction, how="left")
    to_rename = {
        "PUBLISHED_DATE": "LAST_INTERACTION_DATE",
        "CONTENT_URL": "LAST_CONTENT_URL_INTERACTION",
        "CONTENT_TITLE": "LAST_CONTENT_TITLE_INTERACTION"
    }
    df = df.rename(columns=to_rename)
    df["LAST_INTERACTION_DATE"] = pd.to_datetime(df["LAST_INTERACTION_DATE"].str[:-5]).dt.strftime("%a. %d %b.")
    df = df.sort_values(by=["INTERACTION_SCORE", "LAST_INTERACTION_DATE"], ascending=[False, False])
    return df.reset_index(drop=True)

df_profiles = get_unique_profile(df_interactions)
print("Profiles:", len(df_profiles))
df_profiles.head(1)

Profiles: 953


,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_URL,PUBLIC_ID,INTERACTION_SCORE,LAST_INTERACTION_DATE,LAST_CONTENT_URL_INTERACTION,LAST_CONTENT_TITLE_INTERACTION
0,Jonathan,La Cruz,Jonathan La Cruz,Information Technology & Business intelligence...,https://www.linkedin.com/in/ACoAAAlYSwQBS9GBiC...,jonathanlacruz,80,Sun. 03 Dec.,https://www.linkedin.com/feed/update/urn:li:ac...,"I would even say “Good AI requires clean data,..."


### Update Ref Contact

In [9]:
prompt_company = """
I will give you the occupation from a profile I get from LinkedIn, you will return the company you can extract from by checking the word after 'at' or '@'.
If you don't find it return "NA"
Don't put the results into quotes.
"""

prompt_icp = """
I have 2 ideal customer profile, one is a 'data producer' with basic knowledge of Python that could use our Notebook templates to create plugins. 
These plugions are then distributed data via our NaasAI Chat interface.
The other one is a 'data consummer' that will enjoy using NaasAI Chat for its basic LLMs integration but also interested in having its own data available, hence work with the data producer. 
I will give you the occupation from a profile I get from LinkedIn, you will return stricly and only one of the following values inside the simple quotes based on the best match 'DataProducer', 'DataConsummer', 'NotICP' or 'NA' if you don't find a plausible match with the first 3 values.
Don't put the results into quotes.
"""

def get_interactions_by_profile(
    df_init,
    contacts
):
    # Init
    df = df_init.copy()
    interactions = {}
    
    # Cleaning
    to_select = [
        "PROFILE_URL",
        "CONTENT_TITLE",
        "CONTENT_URL",
        "INTERACTION",
        "INTERACTION_CONTENT"
    ]
    df = df[to_select].sort_values(by="PROFILE_URL").reset_index(drop=True)
    df["INTERACTION_TEXT"] = ""
    df.loc[df["INTERACTION"] == "POST_REACTION", "INTERACTION_TEXT"] = df["INTERACTION_CONTENT"].str.capitalize() + " '" + df["CONTENT_TITLE"].str.strip() + "' (" + df["CONTENT_URL"] + ")"
    df.loc[df["INTERACTION"] == "POST_COMMENT", "INTERACTION_TEXT"] = "Comment '" + df["INTERACTION_CONTENT"].str.capitalize() + "' on '" + df["CONTENT_TITLE"].str.strip() + "' (" + df["CONTENT_URL"] + ")"

    # Create interactions by profile
    for contact in contacts:
        tmp_df = df.copy()
        tmp_df = tmp_df[tmp_df["PROFILE_URL"] == contact].reset_index(drop=True)
        interests = ""
        for row in tmp_df.itertuples():
            interaction_text = row.INTERACTION_TEXT
            interests = f"{interests}{interaction_text}, "
        interactions[contact] = interests.strip()
    return interactions

def remove_emojis(text):
    # Emoji pattern
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Remove emojis from the text
    text = emoji_pattern.sub(r'', text)
    return text.strip()

def update_contacts_info(
    df_init,
    df_leads,
    df_contacts,
    input_dir,
):
    # Init
    df = df_init.copy()

    # Filter on profile and concat with leads init
    df = df[df["PROFILE_URL"].str.contains("https://www.linkedin.com/in/.+")]
    df = df.sort_values(by=["INTERACTION_SCORE"], ascending=[False]).reset_index(drop=True)
    df = pd.concat([df_leads, df]).drop_duplicates("PROFILE_URL").fillna("TBD")

    # Cleaning: Remove emojis from name and occupation
    df["FIRSTNAME"] = df.apply(lambda row: remove_emojis(row["FIRSTNAME"]), axis=1)
    df["LASTNAME"] = df.apply(lambda row: remove_emojis(row["LASTNAME"]), axis=1)
    df["OCCUPATION"] = df.apply(lambda row: remove_emojis(row["OCCUPATION"]), axis=1)
    df["FULLNAME"] = df["FIRSTNAME"] + " " + df["LASTNAME"]

    # Enrich: Additional column: ICP, company and notes
    if not "ICP" in df.columns:
        df["ICP"] = "TBD"
    if not "COMPANY" in df.columns:
        df["COMPANY"] = "TBD"
    if not "CRM_CONTACT_ID" in df.columns:
        df["CRM_CONTACT_ID"] = "TBD"
        
    # Create notes from interactions
    leads = df["PROFILE_URL"].unique()  
    df["NOTES"] = df["PROFILE_URL"].map(get_interactions_by_profile(df_interactions, leads))
    
    # Find ICP and Company
    for row in df.itertuples():
        index = row.Index
        fullname = row.FULLNAME
        occupation = row.OCCUPATION
        icp = row.ICP
        company = row.COMPANY
        profile_url = row.PROFILE_URL
        profile_id = profile_url.split("/")[-1]
        public_id = row.PUBLIC_ID
        crm_contact = row.CRM_CONTACT_ID

        # Get ICP
        if icp == "TBD":
            icp = create_chat_completion(openai_api_key, prompt_icp, occupation)
            print(f"{index+1} - 🧑‍💼 '{fullname}' ICP: {icp} ({occupation})")
            df.loc[index, "ICP"] = icp.replace("'", "")
            time.sleep(2)

        # Update Company info
        if company == "TBD":
            company = create_chat_completion(openai_api_key, prompt_company, occupation)
            print(f"{index+1} - 🏢 '{fullname}' Company: {company} ({occupation})")
            df.loc[index, "COMPANY"] = company.replace("'", "")
            time.sleep(2)

        # Find if interaction PROFILE_ID or PUBLIC_ID match with CRM LinkedIn URL then get company name
        if crm_contact == "TBD":
            for x in [profile_id, public_id]:
                tmp_df = df_contacts[df_contacts["LINKEDINBIO"].astype(str).str.contains(x)].reset_index(drop=True)
                if len(tmp_df) > 0:
                    break

            if len(tmp_df) > 0:
                crm_contact_id = ", ".join(tmp_df["CONTACT_ID"].astype(str).unique().tolist())
                if company == "TBD" or company == "NA":
                    crm_companies = ", ".join(tmp_df["COMPANY"].unique().tolist())
                    print(f"{index+1} - 🏢 '{fullname}' CRM Company: {crm_companies} (Company: {company})")
                    df.loc[index, "COMPANY"] = crm_companies
            else:
                crm_contact_id = "NA"
            df.loc[index, "CRM_CONTACT_ID"] = crm_contact_id            
    return df.reset_index(drop=True)
    
df_leads_new = update_contacts_info(
    df_profiles,
    df_leads,
    df_contacts,
    input_dir,
)  
print("Leads updated:", len(df_leads_new))
df_leads_new.head(5)

6 - 🏢 'William Wied' Company: NA (AI Augmented - Growth Insights & Business Optimization)
17 - 🏢 'Marc Cavazza' Company: NA (Professor of AI (posting in a personal capacity))
21 - 🏢 'Peter Jeitschko' Company: NA (I help organizations to automate repetitive task with Artificial Intelligence. Anti-AI regulation activist.)
27 - 🏢 'Harim M.' Company: NA (I am a Shark)
39 - 🏢 'Andrejs Karpovs' Company: NA (I help leaders and individual contributors navigate the AI era)
44 - 🏢 'Hanane DUPOUY' Company: NA (Algorithmic Trader, CFA | AI Passionate)
48 - 🏢 'Davide Imperati, PhD, CMath, MIMA' Company: NA (Data Engineering, Artificial Intelligence & Data Science, Cloud Migration, Quant Development)
52 - 🏢 'Jiajun (Jeff) Lu' Company: NA (Make the world more intelligent and happy.)
54 - 🏢 'Ani Panda' Company: NA (Digital Transformation Leadership | Data Strategy, Architecture, Engineering, Governance & Integration | Startup Enthusiast)
56 - 🏢 'Emmanuel Oluga' Company: NA (Actualising the Future of A

KeyError: 'LINKEDINBIO'

In [ ]:
# def get_linkedin_url(company):
#     # Init linkedinbio
#     linkedinbio = "UNKNOWN"

#     # Create query
#     query = f"{company.replace(' ', '+')}+LinkedIn+company"
#     print("Google query: ", query)

#     # Search in Google
#     for i in search(query, tld="com", num=10, stop=10, pause=2):
#         pattern = "https:\/\/.+.linkedin.com\/company\/.([^?])+"
#         result = re.search(pattern, i)

#         # Return value if result is not None
#         if result != None:
#             linkedinbio = result.group(0).replace(" ", "")
#             time.sleep(2)
#             return linkedinbio
#     return linkedinbio

# def update_ref_company(
#     df_init,
#     df_contact,
#     ref_company,
#     input_dir
# ):
#     # Init
#     df = df_init.copy()
            
#     # Get companies from direct interactions
#     df_company_d = df_init.copy()
#     df_company_d = df_company_d[df_company_d["PROFILE_URL"].str.contains("https://www.linkedin.com/company/.+")]
#     to_keep = [
#         "FULLNAME",
#         "PROFILE_URL",
#         "INTERACTION_SCORE"
#     ]
#     to_rename = {
#         "FULLNAME": "COMPANY_NAME",
#         "PROFILE_URL": "LINKEDIN_URL",
#         "INTERACTION_SCORE": "DIRECT_INTERACTIONS"
#     }
#     df_company_d = df_company_d[to_keep].rename(columns=to_rename)
    
#     # Get companies from indirect interactions
#     df_company_i = df_contact.copy()
#     to_group = [
#         "COMPANY",
#     ]
#     to_agg = {
#         "INTERACTION_SCORE": "sum"
#     }
#     to_rename = {
#         "COMPANY": "COMPANY_NAME",
#         "INTERACTION_SCORE": "INDIRECT_INTERACTIONS"
#     }
#     df_company_i = df_company_i.groupby(to_group, as_index=False).agg(to_agg).rename(columns=to_rename)
    
#     # Concat
#     fillna = {
#         "LINKEDIN_URL": "TBD",
#         "DIRECT_INTERACTIONS": 0,
#         "INDIRECT_INTERACTIONS": 0
#     }
#     df_company = pd.concat([ref_company, df_company_d, df_company_i]).drop_duplicates("COMPANY_NAME").fillna(fillna)
#     df_company["INTERACTION_SCORE"] = df_company["DIRECT_INTERACTIONS"] * 5 + df_company["INDIRECT_INTERACTIONS"]
#     df_company = df_company.sort_values(by=["INTERACTION_SCORE"], ascending=[False])
#     df_company = df_company[df_company["COMPANY_NAME"] != "TBD"].reset_index(drop=True)
#     for row in df_company.itertuples():
#         index = row.Index
#         company_id = row.COMPANY_ID
#         company_name = row.COMPANY_NAME
#         linkedin_url = row.LINKEDIN_URL
#         interaction_score = row.INTERACTION_SCORE
#         if linkedin_url == "TBD":
#             print(f"{index} - Find LinkedIn URL for '{company_name}'")
#             linkedin_url = get_linkedin_url(company_name)
#             print("LinkedIn URL:", linkedin_url)
#             df_company.loc[index, "LINKEDIN_URL"] = linkedin_url
#             pdump(input_dir, df_company, "companies")
            
#         if "company" in linkedin_url and interaction_score >= 3 and str(company_id) == "None":
#             print(f"{index} - Update LinkedIn data '{company_name}':", linkedin_url, f'({int(interaction_score)})')
#             tmp_df = linkedin.connect(li_at, JSESSIONID).company.get_info(linkedin_url)
#             if len(tmp_df) > 0:
#                 company_name = tmp_df.loc[0, "COMPANY_NAME"].replace(" ", "_").lower()
#                 pdump(input_dir, tmp_df, f"{company_name}_linkedin_company")
#                 df_company.loc[index, "COMPANY_ID"] = tmp_df.loc[0, "COMPANY_ID"]
#                 df_company.loc[index, "COMPANY_NAME"] = tmp_df.loc[0, "COMPANY_NAME"]
#                 df_company.loc[index, "LINKEDIN_URL"] = tmp_df.loc[0, "COMPANY_URL"]
#                 df_company.loc[index, "INDUSTRY"] = tmp_df.loc[0, "INDUSTRY"]
#                 df_company.loc[index, "STAFF_COUNT"] = tmp_df.loc[0, "STAFF_COUNT"]
#                 df_company.loc[index, "STAFF_RANGE"] = tmp_df.loc[0, "STAFF_RANGE"]
#                 df_company.loc[index, "FOLLOWER_COUNT"] = tmp_df.loc[0, "FOLLOWER_COUNT"]
#                 df_company.loc[index, "COUNTRY"] = tmp_df.loc[0, "COUNTRY"]
#                 df_company.loc[index, "CITY"] = tmp_df.loc[0, "CITY"]
#                 df_company.loc[index, "WEBSITE"] = tmp_df.loc[0, "WEBSITE"]
#                 df_company.loc[index, "TAGLINE"] = tmp_df.loc[0, "TAGLINE"]
#                 df_company.loc[index, "DESCRIPTION"] = tmp_df.loc[0, "DESCRIPTION"]
#     return df_company.reset_index(drop=True)

# df_company = update_ref_company(
#     df_profiles,
#     df_contact,
#     ref_company,
#     input_dir
# )
# print("Companies:", len(df_company))
# df_company.head()

## Output

### Send "Leads" to spreadsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(data=df, sheet_name=ref_leads_name, append=False)

### Send "Companies" to spreadsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(data=df_company, sheet_name=ref_company_name, append=False)